In [1]:
# Needed to shuffle the deck of cards
import random

#Permanent variables
suits = ('Hearts','Diamonds','Clubs','Spades')
ranks = ('Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack','Queen','King','Ace')
values = {'Two':2,'Three':3,'Four':4,'Five':5,'Six':6,'Seven':7,'Eight':8,'Nine':9,'Ten':10,'Jack':10,'Queen':10,'King':10,'Ace':11}

class Card:
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    # Show the rank and the suit of a card when printing it
    def __str__(self):
        return f'{self.rank} of {self.suit}'    

class Deck:
    # Make a deck using the Card class
    def __init__(self):
        # The cards in the deck
        self.all_cards = []
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit,rank))
    def shuffle(self):
        random.shuffle(self.all_cards)
    # Remove one card from the deck
    def deal_one(self):
        return self.all_cards.pop()
    # Check the cards in the deck by printing them and the total number of cards
    def __str__(self):
        for card in self.all_cards :
            print(card)
        return f'{len(self.all_cards)}'

deck = Deck()
deck.shuffle()

class Player:
    def __init__(self):
        # The cards in the hand of the player
        self.all_cards = []
        # The total value of the cards in the hand
        self.value = 0
        # Number of aces in the hand
        self.aces = 0
    # Adding cards to the hand
    def add_card(self,new_card):
        self.all_cards.append(new_card)
    # Ace can have a value of 1 or 11
    def adjust_for_ace(self):
        # If the value exceeds 21 then we will consider the value of Ace equal to 1. (11-1)*(number of aces)
        if self.value > 21 :
            self.value = self.value - (10*self.aces)
        return self.value

player_one = Player()
dealer = Player()

class Chips:
    def __init__(self):
        self.total = 100
        self.bet = 0
    def __str__(self):
        return f'{self.total}'
    def win_bet(self):
        self.total += self.bet 
        return self.total
    def lose_bet(self):
        self.total -= self.bet
        return self.total

bankroll = Chips()

# Function for taking bets, a Chips class must be created before calling this function
def take_bet(bankroll):
    print(f'You have {bankroll.total} available chips')
    bankroll.bet = input('Please place a bet:')
    # Conditions for the input bet to be an integer and lower than the total chips
    while bankroll.bet.isdigit() == False or int(bankroll.bet) > int(bankroll.total) :
        bankroll.bet = input('Please place a bet:') 
    bankroll.bet = int(bankroll.bet)
    return bankroll.bet

# At the start of each game the player and the dealer gets two cards from the deck
def game_start(player,dealer,deck):
    for x in range(2):
        player.add_card(deck.deal_one())
        dealer.add_card(deck.deal_one())

# The player will be asked to hit while the dealer will always hit if it's cards value is less than 17   
def hit(player,deck):
    player.add_card(deck.deal_one())
    
playing = True    

def hit_or_stand(player,deck):
    # This will control the while loop for turns
    global playing
    # Player decision
    decision = input('Enter H to hit or S to stand.What would you like to do?')
    # Input validation
    while decision not in ['h','s','H','S'] :
        decision = input('Enter H to hit or S to stand.What would you like to do?') 
    else:
        if decision in ['h','H'] :
            print('Player will hit')
            hit(player,deck)
        elif decision in ['s','S'] :
            print('Player will stand')
            playing = False

# This function shows all cards of the player and the dealer's cards without the first one
def show_some(player,dealer):
    player.value = 0
    player.aces = 0
    dealer.value = 0
    for card in player.all_cards:
        # Check if there are any aces in the hand
        if card.rank == 'Ace' :
            player.aces += 1
        player.value += card.value
        print(card)
    # Adjusting the value of aces
    player.adjust_for_ace()
    print(f"The total value of the player's cards is {player.value}")
    # The dealer's hidden card
    print('Unknown Card')
    for card in dealer.all_cards[1:]:
        dealer.value += card.value
        print(card)
    print(f"The total value of the dealer's cards is {dealer.value}")    

# This function shows all of the cards
def show_all(player,dealer):
    player.value = 0
    player.aces = 0
    dealer.value = 0
    dealer.aces = 0
    for card in player.all_cards:
        # Check if there are any aces in the hand
        if card.rank == 'Ace' :
            player.aces += 1
        player.value += card.value
        print(card)
    # Adjusting the value of aces
    player.adjust_for_ace()
    print(f"The total value of the player's cards is {player.value}")
    for card in dealer.all_cards:
        # Check if there are any aces in the hand
        if card.rank == 'Ace' :
            dealer.aces += 1
        dealer.value += card.value
        print(card)
    # Adjusting the value of aces
    dealer.adjust_for_ace()
    print(f"The total value of the dealer's cards is {dealer.value}")

# End of the game scenarios
def player_busts(bankroll):
    print('You have busted.\nYou lost the bet.')
    print(f'You have {bankroll.lose_bet()} chips left')
    return bankroll
def player_wins(bankroll):
    print('Congratulations,you won!')
    print(f'You have {bankroll.win_bet()} chips left')
    return bankroll
def dealer_busts(bankroll):
    print('Dealer busted.\nCongratulations,you won!')
    print(f'You have {bankroll.win_bet()} chips left')
    return bankroll
def dealer_wins(bankroll):
    print('Dealer won.\nYou lost the bet.')
    print(f'You have {bankroll.lose_bet()} chips left')
    return bankroll
def push(bankroll):
    print("Dealer's hand is equal to player's hand.\nIt is a push.")
    print(f'You have {bankroll} chips left')
    return bankroll

def winner_check(player,dealer,bankroll):
    if player.value > 21 :
          player_busts(bankroll)
    elif dealer.value > 21 :
          dealer_busts(bankroll)
    elif 21-player.value < 21-dealer.value :
          player_wins(bankroll)
    elif 21-player.value > 21-dealer.value :
          dealer_wins(bankroll)
    elif player.value == dealer.value :
          push(bankroll)

game_on = True

#Function used to play again or exit
def play_again():
    global game_on
    choice = input('Do you want to play again? Y/N :')
    # Input validation,it must be y or n
    while choice.upper() not in ['Y','N'] :
        print('Please enter Y for yes or N for no!')
        choice = input('Do you want to play again? Y/N :')
    if choice.upper() in ['N']:
        print('Game Over')
        game_on = False
    else:
        game_on = True
    
def reset_hands(player,dealer):
    player.all_cards = []
    dealer.all_cards = []
        

In [2]:
player_one = Player()
dealer = Player()
bankroll = Chips()
game_on = True

print('Welcome')
while game_on:
    deck = Deck()
    deck.shuffle()
    reset_hands(player_one,dealer)
    # Player must place a bet
    take_bet(bankroll)
    print(f'Your bet is {bankroll.bet}.\nThe blackjack game has started')
    # Each player gets 2 cards from the deck
    game_start(player_one,dealer,deck)
    # Show player's cards and dealer's cards without the first one
    show_some(player_one,dealer)
    # Used to control the player/dealer turn while loop
    playing = True
    # Player's turn
    while player_one.value <= 21 and playing == True :
        hit_or_stand(player_one,deck)
        show_some(player_one,dealer)
    # Dealer's turn
    while playing == False :
        show_all(player_one,dealer)
        if dealer.value < 17 :
            hit(dealer,deck)
        else:
            break
    winner_check(player_one,dealer,bankroll)
    play_again()
            

Welcome
You have 100 available chips
Please place a bet:10
Your bet is 10.
The blackjack game has started
Four of Clubs
Queen of Hearts
The total value of the player's cards is 14
Unknown Card
Ten of Diamonds
The total value of the dealer's cards is 10
Enter H to hit or S to stand.What would you like to do?h
Player will hit
Four of Clubs
Queen of Hearts
Ace of Spades
The total value of the player's cards is 15
Unknown Card
Ten of Diamonds
The total value of the dealer's cards is 10
Enter H to hit or S to stand.What would you like to do?h
Player will hit
Four of Clubs
Queen of Hearts
Ace of Spades
Three of Spades
The total value of the player's cards is 18
Unknown Card
Ten of Diamonds
The total value of the dealer's cards is 10
Enter H to hit or S to stand.What would you like to do?s
Player will stand
Four of Clubs
Queen of Hearts
Ace of Spades
Three of Spades
The total value of the player's cards is 18
Unknown Card
Ten of Diamonds
The total value of the dealer's cards is 10
Four of Clu